In [3]:
# 🤖 General Health Chatbot using FLAN-T5-Large
# Task: Respond to general health questions safely
# Model: google/flan-t5-large (fast, public, accurate)


# 🛠️ Step 1: Install required library
!pip install -q transformers

# 🔄 Step 2: Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# ✅ Step 3: Load the FLAN-T5-Large model and tokenizer
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto" if torch.cuda.is_available() else "cpu"
)

# 🛡️ Step 4: Define the safety filter to block sensitive/risky queries
def is_safe_query(user_input):
    blocked_keywords = [
        "suicide", "kill", "overdose", "abortion", "self-harm", "emergency",
        "depression", "hurt myself", "pills", "cut", "hang", "bleeding",
        "faint", "heart attack", "chest pain", "unconscious", "drug", "alcohol",
        "poison", "hopeless", "injury", "stab", "expired medicine", "stop breathing"
    ]
    return not any(word in user_input.lower() for word in blocked_keywords)

# 🤖 Step 5: Define the response generation function
def get_response(user_input):
    # Check if the query is safe to respond to
    if not is_safe_query(user_input):
        return "⚠️ Sorry, I can't help with that. Please consult a licensed medical professional."

    # Format the prompt to guide the model's tone and style
    prompt = f"Answer this like a helpful health assistant: {user_input}"

    # Tokenize the input for the model
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate response using the model
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        pad_token_id=tokenizer.eos_token_id  # Prevent padding warning
    )

    # Decode the output and clean it up
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

# 💬 Step 6: Start interactive chatbot loop
print("🤖 General Health Chatbot (FLAN-T5-Large)\nType 'exit' to quit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Stay healthy! 🌿")
        break

    response = get_response(user_input)
    print("Chatbot:", response, "\n")


🤖 General Health Chatbot (FLAN-T5-Large)
Type 'exit' to quit.

You: How often should I exercise?
Chatbot: Exercise for at least 30 minutes every day. 

You: Why do we get sore throats?
Chatbot: a cold 

You: Why is washing hands important?
Chatbot: Keeping your hands clean can help prevent illness 

You: "What to do during a heart attack?"
Chatbot: ⚠️ Sorry, I can't help with that. Please consult a licensed medical professional. 

You: "Can I overdose on painkillers?"
Chatbot: ⚠️ Sorry, I can't help with that. Please consult a licensed medical professional. 

You: exit
Chatbot: Stay healthy! 🌿
